<a href="https://colab.research.google.com/github/vinurajd/google_collab_DL/blob/master/Fashion_MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math 
import numpy
import logging
import matplotlib.pyplot as plt

In [130]:
tfds.disable_progress_bar()

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [131]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_data, test_data = dataset['train'], dataset['test']

In [132]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [ ]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

In [134]:
def normalize_img(img, labels):
  img = tf.cast(img,tf.float32)/255
  return img, labels

In [135]:
train_data=train_data.map(normalize_img)
test_data=test_data.map(normalize_img)

train_data=train_data.cache()
test_data=test_data.cache()

In [ ]:
i=0
plt.figure(figsize=(10,10))
for (img, label) in test_data.take(25):
  img=img.numpy().reshape((28,28))
  plt.xticks([])
  plt.yticks([])
  plt.subplot(5,5,i+1)
  plt.imshow(img,cmap=plt.cm.binary)
  plt.xlabel(class_names[label])
  i+=1

plt.show()

In [137]:
cnn_model = tf.keras.Sequential([
tf.keras.layers.Conv2D(128,(3,3),padding='same',activation=tf.nn.relu,input_shape=(28,28,1)),
tf.keras.layers.MaxPooling2D((2,2),strides=2),
tf.keras.layers.Conv2D(256,(3,3),activation=tf.nn.relu,padding='same'),
tf.keras.layers.MaxPooling2D((2,2),strides=2),
tf.keras.layers.Conv2D(512,(3,3),activation=tf.nn.relu,padding='same'),
tf.keras.layers.MaxPooling2D((2,2),strides=2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128,activation=tf.nn.relu),
tf.keras.layers.Dense(10,activation=tf.nn.softmax)                                 
])

In [138]:
cnn_model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 28, 28, 128)       1280      
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 14, 14, 256)       295168    
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 7, 7, 512)         1180160   
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 3, 3, 512)         0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 4608)            

In [139]:
cnn_model.compile(loss=tf.losses.SparseCategoricalCrossentropy(),
                  optimizer="adam",
                  metrics=['accuracy'])

In [140]:
BATCH_SIZE=32
train_data=train_data.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_data=test_data.cache().batch(BATCH_SIZE)

In [141]:
cnn_model.fit(train_data,epochs=10,steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Epoch 1/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3787 - accuracy: 0.8626
Epoch 2/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2382 - accuracy: 0.9114
Epoch 3/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1920 - accuracy: 0.9288
Epoch 4/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1576 - accuracy: 0.9419
Epoch 5/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.1303 - accuracy: 0.9520
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1048 - accuracy: 0.9610
Epoch 7/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0933 - accuracy: 0.9656
Epoch 8/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0757 - accuracy: 0.9720
Epoch 9/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0674 - accuracy: 0.9754
Epoch 10/10
1875/1875 [==============================] - 13s 7ms/step - l

In [142]:
test_loss, test_accuracy = cnn_model.evaluate(test_data, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

313/313 [==============================] - 2s 7ms/step - loss: 0.3484 - accuracy: 0.9186
Accuracy on test dataset: 0.9186000227928162
